In [ ]:
import numpy as np
import pandas as pd
import glob
import os

from sklearn.model_selection import train_test_split
 
import tensorflow as tf
import keras as ks
from keras.models import Sequential
from keras.layers import Dense, Input, GRU, LSTM

from keras.callbacks import EarlyStopping, CSVLogger, TerminateOnNaN
import json

import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [ ]:
from weightedValues import weightValues

In [ ]:
def OpenJsontoArr(path):
    file = open(path)           #"X:\\KI Praktikum\\validate_Data\\3darray_x_cv.json"
    x_3d = json.load(file)
    file.close()
    x_3d = np.asarray(x_3d)
    return x_3d

In [ ]:
def get_lstm_model(layers=[5, 5, 5], dropout=0, activation = "relu", input_shape = (1, 1), loss = "mean_squared_error"):
    
    model = Sequential()
    model.add(Input(input_shape))
    
    for i in range(len(layers)):
        if i == len(layers)-1:
            model.add(LSTM(layers[i], stateful = False, dropout=dropout, activation = activation, return_sequences=False))
        else:
            model.add(LSTM(layers[i], stateful = False, dropout=dropout, activation = activation, return_sequences=True))
        #model.add(Dropout(dropout))

    model.add(Dense(1, activation="linear"))
    # compile the model
    model.compile(
        loss= loss,
        #metrics=[ "max_loss"],
        optimizer = tf.keras.optimizers.Adamax())
    # return compiled model
    return model

In [ ]:
x_files = glob.glob(os.path.join("X:\\KI Praktikum\\validate_Data\\3darrays", "*15_x.json"))
y_files = glob.glob(os.path.join("X:\\KI Praktikum\\validate_Data\\3darrays", "*15_y.json"))
x_cv = OpenJsontoArr(x_files[0])
y_cv = OpenJsontoArr(y_files[0])
y_cv = y_cv.flatten()


In [ ]:
print(x_cv[:10, :])

In [ ]:
df_y = pd.DataFrame({"y": y_cv})
print(df_y.head(15))
df_weights = weightValues(df_y, weightMin=1, weightMax=2, namey="y", nameWeights="weights")

In [ ]:
#df_weights = df_weights.drop("0", axis=1)
print(df_weights.head(10))
# x_weighted = np.zeros((x_cv.shape[0], x_cv.shape[1], x_cv.shape[2]+1))
# x_weighted[:,:,:6] = x_cv
# x_weighted[:,:,-1] = df_weights
y_cv = np.array(df_weights)
y_cv = y_cv[:, :2]

In [ ]:
"""Custom loss function to reduce the absolute error of the individual sample as well as the 
maximum error"""
def customLoss(y_true, y_pred):
    weight = 1.6
    difference = tf.abs(y_true - y_pred)
    exponent = tf.exp(tf.multiply(weight, difference))
    weighted_muls = tf.multiply(difference, exponent)
    boltzmann_op = tf.reduce_sum(weighted_muls) / tf.reduce_sum(exponent)
    loss = tf.add(boltzmann_op, tf.losses.mean_absolute_error(y_true, y_pred))

    return loss

In [ ]:
#losses = [customLoss()]
#,          tf.keras.losses.MeanSquaredError()] #,
#           tf.keras.losses.MeanAbsoluteError(),
#           tf.keras.losses.CosineSimilarity()]
x_lstm_train, x_lstm_test, y_lstm_train, y_lstm_test = train_test_split(x_cv, y_cv, shuffle = False, test_size=0.1)

earlyStop = EarlyStopping(monitor='val_loss', patience=100, verbose =1, mode = "auto")
csvLogger = CSVLogger('X:\\KI Praktikum\\csvLoggerCustomLoss.xlsx')
stopNaN = TerminateOnNaN()

In [ ]:
#i = 0
#summary = np.array(len(losses))
#for loss in losses:
lstmmodel = get_lstm_model(layers=[1], dropout=0, activation = "elu", input_shape = (15,6), loss = customLoss)

summary = lstmmodel.fit(x_lstm_train, y_lstm_train[:,0], validation_data=(x_lstm_test, y_lstm_test), sample_weight = y_lstm_train[:,1], 
                        epochs = 500, batch_size = 4000, callbacks=[earlyStop, csvLogger, stopNaN], shuffle = False)

y_pred = lstmmodel.predict(x_cv)

difference = y_cv[:, 0] - y_pred.flatten()
error_avg = np.mean(abs(difference))
error_max = max(abs(difference))
#mean_absolute = tf.keras.losses.mean_absolute_error(y_cv[:,0], y_pred.flatten())
#print("mean absolute: ", mean_absolute)
print("max: ", error_max)
print("mean abs error: ", error_avg)
#i+=1

In [ ]:
scatter_mode = "lines"
train_loss = summary.history["loss"]
validation_loss = summary.history["val_loss"]

fig= make_subplots(rows=1, cols=1, shared_xaxes= True, print_grid= True)

fig.add_trace(go.Scatter( y= train_loss, name= 'training loss', mode= scatter_mode), row= 1, col= 1)
fig.add_trace(go.Scatter( y= validation_loss, name= 'validation loss', mode= scatter_mode), row= 1, col= 1)
fig.update_yaxes(title_text= 'loss', row= 1, col= 1)

fig.update_layout(height=600, width=1200, title_text="Training loss curve")
fig.show()

In [ ]:
modelPath = "X:\\KI Praktikum\\model\\23_09_01lstm_CustomLoss_max12_8.h5"
lstmmodel.save(modelPath)
